# CO Stripping Integration
Linn Kelley<br>
08/26/2022

This script calculates electrochemically active surface area (ECSA) for a catalyst using the CO stripping method.

CO stripping is used to measure catalyst accessibility (ECSA). CO has a high affinity for Pt and will displace other adsorbed species on catalyst particles. CO stripping is performed using a cyclic voltammogram (CV) with multiple cycles between 0 V and 1 V vs RHE after a CO monolayer forms on the catalyst surface. CO is oxidized off of the catalyst surface between 0.5 and 0.9 V during the first anodic sweep (and in small amounts during the following anodic sweeps):<br>
<center>
$CO^*+H_2O\rightleftharpoons CO_2+2H^++2e^-$<br>
</center>
The characteristic surface charge expended to oxidize CO is 420 µC/cm<sup>2</sup> which is used to convert integrated charge to ECSA. This is calculated by integrating the first oxidation peak with respect to time: 
<center>
$Q=\int Idt$
</center>
It is important to bound the left side of the peak at the capacitive baseline current observed in subsequent CV cycles. The right side of the peak should be bound on the right side of the oxidation peak prior to the broad peak corresponding to Pt oxide formation. The function integrates between the first and second CV cycles. 

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

### Import file

In [2]:
co = pd.read_csv('Sample CO stripping file.txt',delimiter='\t',header=14)

### Add a time column to the dataframe 
If your file does not have a time column, use the addtime function. 
addtime has two inputs, a dataframe (df) and the interval at which data points were collected (nu). The data collection rate can be found by opening your text file in Notepad and dividing 1/Waveform Rate (Hz).

In [3]:
def addtime(df,nu):
    time = []
    for i in range(len(df)):
        t=i*nu
        time.append(t)
    df['Time']=time 
        
addtime(co,0.01)

### co_stripping inputs
df: dataframe with voltage, current, and time column<br>
c: color<br>
v_max: maximum voltage of CV (V)
xlim: x-limits of the plot in format [x1,x2]<br>
ylim: y-limits of the plot in format [y1,y2]<br>
title: title of the plot and how the image will be saved<br>
area: cell area (cm<sup>2</sup>)<br>
CL: catalyst loading (mg<sub>Pt</sub>/cm<sup>2</sup>)<br>

The function requires user selection of the left and right side of the integration region. The user should select these points in the order:<br>
1. Left side of oxidation peak
2. Right side of oxidation peak

<img src="CO guide.png" width="600">

When the mapping image pops up, the magnifying glass can be used to zoom in as needed while selecting points. The home button will bring user back to the original image. Hover the mouse over the desired point and press the space bar to select a point. A red X will appear on the point. To erase the point and try again, press the backspace or delete key. When both points have been selected, press the enter key to continue. 

### co_stripping returns
[q,ecsa]<br>
q: integrated charge (mC/cm<sup>2</sup>)<br>
ecsa: ECSA (m<sup>2</sup>/g<sub>Pt</sub>)

### co_stripping function

In [4]:
def co_stripping(df,c,v_max,xlim,ylim,title,area,CL):
    
    # CV data is plotted and user specifies integration points. 
    plt.plot(df['Volts'],df['Amps']*(1000/area),color=c)
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.title(title)
    plt.xlabel('Cell Voltage (V vs SHE)')
    plt.ylabel('Current Density (mA/cm$^2$)')
    pts = plt.ginput(n=-1,timeout=-1,mouse_add=None,mouse_pop=None,mouse_stop=None)
    
    # The data indicies closest to the selected points are recorded in the variable ind. 
    ind = []
    right_bound = np.where(v_max-df['Volts']<0.005)[0][0]
    points = list(zip(df['Volts'][:right_bound],df['Amps'][:right_bound]*1000/area))
    def distance(a,b):
        return(sum([(k[0]-k[1])**2 for k in zip(a,b)])**0.5)
    for i,pt in enumerate(pts):
        dists = [distance([pt[0], pt[1]],k) for k in points]
        ind.append(dists.index(min(dists)))
        
    # The voltage, time, and current columns are trimmed to the integration region.    
    v = df['Volts'][ind[0]:ind[1]+1]
    t = df['Time'][ind[0]:ind[1]+1]
    j = df['Amps'][ind[0]:ind[1]+1]*1000/area
    
    # The integration region is selected for the second CV cycle.
    post_peak = np.where(df['Volts']>df['Volts'][ind[1]])   
    for i in range(len(post_peak[0])):
        if (post_peak[0][i+1]-post_peak[0][i])!=1:
            cycle = post_peak[0][i+1]-1-ind[1]
            break           
    j2 = df['Amps'][ind[0]+cycle:ind[1]+cycle+1]*1000/area
    
    # The peak is integrated between the first and second CV cycles.
    g_pt = CL*area/1000
    q = round((np.trapz(j,t) - np.trapz(j2,t)),2)    
    ecsa = round((np.trapz(j,t) - np.trapz(j2,t))*area/(1000*420e-6*10000*g_pt),2)
    print(q)
    print(ecsa)
    
    # The integration region is shaded and the figure is saved.
    plt.fill_between(v,j2,j,color=c,alpha=0.5)
    plt.savefig(title+'.png',format='png')
    return q,ecsa

### Example use

In [5]:
s = co_stripping(co,'mediumblue',1,[0,1.1],[-5,35],'test',50,0.264)

42.79
38.59


In [6]:
s

(42.79, 38.59)

CO-stripping integration requires a little CO-llaboration!